In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path
import os
from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer_speedup import train_model

from torch.optim.lr_scheduler import StepLR

if __name__ == '__main__':
    # CUDA 성능 플래그 최적화
    torch.backends.cudnn.benchmark = True
    # TF32 텐서 코어 사용을 허용하여 Ampere 아키텍처 이상 GPU에서 연산 속도 향상
    torch.backends.cuda.matmul.allow_tf32 = True
    
    # 설정값 정의
    # 장치 설정: 사용 가능한 경우 GPU(cuda)를, 그렇지 않으면 CPU를 사용
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")
    
    DATA_DIR = Path("./datasets/korean_emotion_complex_vision_10_percent_verified_processed")
    # 사용하고자 하는 모델 하나만 남기고 다른 MODEL_NAME 앞에 # 붙여서 주석처리
    #MODEL_NAME = 'resnet18'            #철원
    #MODEL_NAME = 'resnet50'            #철원
    MODEL_NAME = 'shufflenet_v2'        #철원
    #MODEL_NAME = 'mobilenet_v3_small'  #승현님
    #MODEL_NAME = 'efficientnet_v2_s'   #승현님
    #MODEL_NAME = 'squeezenet'          #승현님
    
    NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정해야 합니다. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
    BATCH_SIZE = 64  # 배치 크기를 늘려 GPU 메모리 사용 최적화
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 100
    EARLY_STOPPING_PATIENCE = 10 # 10번 연속 성능 개선이 없으면 조기 종료
    STEPS_PER_EPOCH = None # 빠른 테스트를 위해 에폭당 배치 수를 제한하려면 숫자로 변경 (예: 100)

    
    # 데이터 증강을 포함한 훈련용 Transform 정의
    #train_transform = transforms.Compose([
    #    transforms.Resize((224, 224)),
    #    transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
    #    transforms.RandomRotation(15),           # -15도 ~ 15도 사이로 랜덤 회전
    #    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # 밝기, 대비, 채도 조절
    #    transforms.ToTensor(),
    #    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    #])
    
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        # TrivialAugmentWide 추가, 이미지에 다양한 변형(자르기, 색상 왜곡, 회전 등)을 알아서 최적의 강도로 적용, 과적합 방지.
        transforms.TrivialAugmentWide(), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    # 증강이 없는 검증/테스트용 Transform 정의
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # 훈련용과 검증용 데이터셋을 각각 생성.
    train_dataset = EmotionDataset(data_dir=DATA_DIR / "train", transform=train_transform)
    val_dataset = EmotionDataset(data_dir=DATA_DIR / "val", transform=val_transform)

    # 데이터로더를 각각 생성. (검증용은 섞을 필요가 없음)
    #train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    #val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # DataLoader I/O 튜닝
    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True,
        # CPU 코어를 최대한 활용하여 데이터를 미리 GPU 메모리로 올리는 작업을 병렬 처리
        num_workers=min(8, os.cpu_count()), 
        pin_memory=True, # GPU로의 데이터 전송 속도 향상
        persistent_workers=True, # 워커 프로세스를 계속 유지하여 오버헤드 감소
        prefetch_factor=2, # 각 워커가 미리 로드할 배치 수
        drop_last=True # 마지막 배치가 배치 사이즈보다 작을 경우 버려서 연산 일관성 유지
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False,
        num_workers=min(8, os.cpu_count()),
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )

    NUM_CLASSES = len(train_dataset.classes)
    
    print("데이터 준비 완료!")
    print(f"훈련 데이터셋 크기: {len(train_dataset)}")
    print(f"클래스 수: {NUM_CLASSES} -> {train_dataset.classes}")

    # 모델, 손실 함수, 옵티마이저 준비
    model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
    # 모델을 지정된 장치로 이동
    model.to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(), 
        weight_decay=1e-4, #과적합 방지를 위한 정규화 기법(Weight Decay), 학습을 방해함으로서 과적합 방지.
        lr=LEARNING_RATE 
        ) 
    scheduler = StepLR(optimizer, step_size=7, gamma=0.1) # 7 에폭마다 학습률을 0.1배로 감소
    
    print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")

    # 모델 훈련 시작
    print("\n모델 훈련을 시작합니다...")
    trained_model = train_model(model, 
                                train_loader, 
                                val_loader, 
                                criterion, 
                                optimizer, 
                                scheduler,
                                DEVICE, 
                                num_epochs=NUM_EPOCHS, 
                                patience=EARLY_STOPPING_PATIENCE,
                                steps_per_epoch=STEPS_PER_EPOCH
                                )

    # 훈련된 모델 저장 (옵션)
    # torch.save(trained_model.state_dict(), f'{MODEL_NAME}_trained.pth')
    # print("훈련된 모델 가중치가 저장되었습니다.")

Using device: cuda
데이터 준비 완료!
훈련 데이터셋 크기: 32407
클래스 수: 7 -> ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
'shufflenet_v2' 모델, 손실 함수, 옵티마이저 준비 완료!

모델 훈련을 시작합니다...
Epoch 1/100
----------
  [Batch 20/506] Train Loss: 1.8385 Acc: 0.2969
  [Batch 506/506] Train Loss: 1.0409 Acc: 0.5781
Train Loss: 1.2677 Acc: 0.5173
Val Loss: 1.1460 Acc: 0.5601

  -> Val Loss 개선됨! (1.1460) 모델 저장.
Epoch 2/100
----------
  [Batch 20/506] Train Loss: 1.1664 Acc: 0.5781
  [Batch 506/506] Train Loss: 0.9024 Acc: 0.6406
Train Loss: 1.0626 Acc: 0.6006
Val Loss: 0.9709 Acc: 0.6317

  -> Val Loss 개선됨! (0.9709) 모델 저장.
Epoch 3/100
----------
  [Batch 20/506] Train Loss: 1.0577 Acc: 0.6250
  [Batch 506/506] Train Loss: 1.0164 Acc: 0.6094
Train Loss: 1.0112 Acc: 0.6192
Val Loss: 0.9610 Acc: 0.6375

  -> Val Loss 개선됨! (0.9610) 모델 저장.
Epoch 4/100
----------
  [Batch 20/506] Train Loss: 1.0106 Acc: 0.6250
  [Batch 506/506] Train Loss: 0.9940 Acc: 0.5938
Train Loss: 0.9746 Acc: 0.6354
Val Loss: 0.9475 Acc: 0.6449

  -> Val Lo

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path
import os
from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer_speedup import train_model

from torch.optim.lr_scheduler import StepLR

if __name__ == '__main__':
    # CUDA 성능 플래그 최적화
    torch.backends.cudnn.benchmark = True
    # TF32 텐서 코어 사용을 허용하여 Ampere 아키텍처 이상 GPU에서 연산 속도 향상
    torch.backends.cuda.matmul.allow_tf32 = True
    
    # 설정값 정의
    # 장치 설정: 사용 가능한 경우 GPU(cuda)를, 그렇지 않으면 CPU를 사용
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")
    
    DATA_DIR = Path("./datasets/korean_emotion_complex_vision_5_percent_verified_processed")
    # 사용하고자 하는 모델 하나만 남기고 다른 MODEL_NAME 앞에 # 붙여서 주석처리
    #MODEL_NAME = 'resnet18'            #철원
    #MODEL_NAME = 'resnet50'            #철원
    MODEL_NAME = 'shufflenet_v2'        #철원
    #MODEL_NAME = 'mobilenet_v3_small'  #승현님
    #MODEL_NAME = 'efficientnet_v2_s'   #승현님
    #MODEL_NAME = 'squeezenet'          #승현님
    
    NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정해야 합니다. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
    BATCH_SIZE = 64  # 배치 크기를 늘려 GPU 메모리 사용 최적화
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 100
    EARLY_STOPPING_PATIENCE = 10 # 10번 연속 성능 개선이 없으면 조기 종료
    STEPS_PER_EPOCH = None # 빠른 테스트를 위해 에폭당 배치 수를 제한하려면 숫자로 변경 (예: 100)

    
    # 데이터 증강을 포함한 훈련용 Transform 정의
    #train_transform = transforms.Compose([
    #    transforms.Resize((224, 224)),
    #    transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
    #    transforms.RandomRotation(15),           # -15도 ~ 15도 사이로 랜덤 회전
    #    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # 밝기, 대비, 채도 조절
    #    transforms.ToTensor(),
    #    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    #])
    
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        # TrivialAugmentWide 추가, 이미지에 다양한 변형(자르기, 색상 왜곡, 회전 등)을 알아서 최적의 강도로 적용, 과적합 방지.
        transforms.TrivialAugmentWide(), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    # 증강이 없는 검증/테스트용 Transform 정의
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # 훈련용과 검증용 데이터셋을 각각 생성.
    train_dataset = EmotionDataset(data_dir=DATA_DIR / "train", transform=train_transform)
    val_dataset = EmotionDataset(data_dir=DATA_DIR / "val", transform=val_transform)

    # 데이터로더를 각각 생성. (검증용은 섞을 필요가 없음)
    #train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    #val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # DataLoader I/O 튜닝
    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True,
        # CPU 코어를 최대한 활용하여 데이터를 미리 GPU 메모리로 올리는 작업을 병렬 처리
        num_workers=min(8, os.cpu_count()), 
        pin_memory=True, # GPU로의 데이터 전송 속도 향상
        persistent_workers=True, # 워커 프로세스를 계속 유지하여 오버헤드 감소
        prefetch_factor=2, # 각 워커가 미리 로드할 배치 수
        drop_last=True # 마지막 배치가 배치 사이즈보다 작을 경우 버려서 연산 일관성 유지
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False,
        num_workers=min(8, os.cpu_count()),
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )

    NUM_CLASSES = len(train_dataset.classes)
    
    print("데이터 준비 완료!")
    print(f"훈련 데이터셋 크기: {len(train_dataset)}")
    print(f"클래스 수: {NUM_CLASSES} -> {train_dataset.classes}")

    # 모델, 손실 함수, 옵티마이저 준비
    model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
    # 모델을 지정된 장치로 이동
    model.to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(), 
        weight_decay=1e-4, #과적합 방지를 위한 정규화 기법(Weight Decay), 학습을 방해함으로서 과적합 방지.
        lr=LEARNING_RATE 
        ) 
    scheduler = StepLR(optimizer, step_size=7, gamma=0.1) # 7 에폭마다 학습률을 0.1배로 감소
    
    print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")

    # 모델 훈련 시작
    print("\n모델 훈련을 시작합니다...")
    trained_model = train_model(model, 
                                train_loader, 
                                val_loader, 
                                criterion, 
                                optimizer, 
                                scheduler,
                                DEVICE, 
                                num_epochs=NUM_EPOCHS, 
                                patience=EARLY_STOPPING_PATIENCE,
                                steps_per_epoch=STEPS_PER_EPOCH
                                )

    # 훈련된 모델 저장 (옵션)
    # torch.save(trained_model.state_dict(), f'{MODEL_NAME}_trained.pth')
    # print("훈련된 모델 가중치가 저장되었습니다.")

Using device: cuda
데이터 준비 완료!
훈련 데이터셋 크기: 17975
클래스 수: 7 -> ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
'shufflenet_v2' 모델, 손실 함수, 옵티마이저 준비 완료!

모델 훈련을 시작합니다...
Epoch 1/100
----------
  [Batch 20/280] Train Loss: 1.8227 Acc: 0.3906
  [Batch 280/280] Train Loss: 1.1848 Acc: 0.5312
Train Loss: 1.3459 Acc: 0.4915
Val Loss: 1.1255 Acc: 0.5906

  -> Val Loss 개선됨! (1.1255) 모델 저장.
Epoch 2/100
----------
  [Batch 20/280] Train Loss: 1.3269 Acc: 0.5625
  [Batch 280/280] Train Loss: 0.9187 Acc: 0.6875
Train Loss: 1.0916 Acc: 0.5912
Val Loss: 0.9945 Acc: 0.6301

  -> Val Loss 개선됨! (0.9945) 모델 저장.
Epoch 3/100
----------
  [Batch 20/280] Train Loss: 1.0729 Acc: 0.5469
  [Batch 280/280] Train Loss: 1.0266 Acc: 0.6094
Train Loss: 1.0133 Acc: 0.6235
Val Loss: 0.9752 Acc: 0.6423

  -> Val Loss 개선됨! (0.9752) 모델 저장.
Epoch 4/100
----------
  [Batch 20/280] Train Loss: 0.8237 Acc: 0.7344
  [Batch 280/280] Train Loss: 0.8736 Acc: 0.6875
Train Loss: 0.9673 Acc: 0.6395
Val Loss: 0.9409 Acc: 0.6537

  -> Val Lo